## 고급 HTML 분석
---

In [ ]:
bs.findAll('table')[4].find_all('tr')[2].find('td').find_all('div')[1].find('a')

### 2.2 다시 BeautifulSoup
---
- 태그 분석하기

In [2]:
<span class="green"></span>
<span class="red"></span>

<span class="red">Heavens! what a virulent attack!</span> replied <span class="green">
    the prince
</span>, not in the least disconcerted by this reception.

SyntaxError: invalid syntax (<ipython-input-2-92cefa8d143c>, line 1)

In [3]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen('http://www.pythonscraping.com/pages/warandpeace.html')
bs = BeautifulSoup(html, 'html.parser')

nameList = bs.findAll('span', {'class': 'green'})
for name in nameList:
    print(name.get_text())

Anna
Pavlovna Scherer
Empress Marya
Fedorovna
Prince Vasili Kuragin
Anna Pavlovna
St. Petersburg
the prince
Anna Pavlovna
Anna Pavlovna
the prince
the prince
the prince
Prince Vasili
Anna Pavlovna
Anna Pavlovna
the prince
Wintzingerode
King of Prussia
le Vicomte de Mortemart
Montmorencys
Rohans
Abbe Morio
the Emperor
the prince
Prince Vasili
Dowager Empress Marya Fedorovna
the baron
Anna Pavlovna
the Empress
the Empress
Anna Pavlovna's
Her Majesty
Baron
Funke
The prince
Anna
Pavlovna
the Empress
The prince
Anatole
the prince
The prince
Anna
Pavlovna
Anna Pavlovna
